# Assigment 04 

In this Assigment, we are going to paarse XBRL files and extract some partes of the files.

The urls from the files are into a xml files, so we need to parse, find the right tags to get the links

In this assigment, insted of saving all files into memory, we will save into disk, extract only the parts we need than save into a dataframe.

## In this version 04.1  I changed the code to extract the tags from XBRL without saving it locally, it will only save our final database

In [1]:
#Importing dependencies
import xml.etree.ElementTree as ET
import pandas as pd
import requests

We can get the list of the files in https://www.sec.gov/structureddata/rss-feeds-submitted-filings

In [2]:
url = 'https://www.sec.gov/Archives/edgar/monthly/xbrlrss-2017-08.xml'

In [3]:
#We get the xml file with the links to ours XBRL files 
xmlfile = requests.get(url).text

In [4]:
#Creating empty list to store the data.
companies = []
urls = []
file = []

In [5]:
#Open the xm file to parse and find the url for the xbrl files
tree = ET.fromstring(xmlfile)
item = tree.findall('channel/item')

#The for loop will parse the xml file to find the link to the xbrl files we need
for entry in item:   
    
    desc = entry.findtext('description')  
    
    if desc == '10-K':
        companyName = ''
        xbrlurl = ''
        for item in entry.iter():

            if item.tag == '{http://www.sec.gov/Archives/edgar}companyName':
                companyName = item.text
                
            if item.tag =='{http://www.sec.gov/Archives/edgar}xbrlFiles':
                for ite  in item:
                    if ite.attrib['{http://www.sec.gov/Archives/edgar}description'] == 'XBRL INSTANCE DOCUMENT':
                        xbrlurl = ite.attrib['{http://www.sec.gov/Archives/edgar}url']
        
        #Some companies dont have the xbrl file we are going to use
        if  xbrlurl != '': 
            companies.append(companyName)  
            urls.append(xbrlurl)
            file.append(xbrlurl.split('/')[-1])

In [6]:
#Creating a Dataframe to easely manipulate the data
data = pd.DataFrame(data ={'Company': companies,
                           'url': urls,
                           'file': file})

In [7]:
#Creating a sample of 100 documents
data = data.sample(100, random_state=42)

In [8]:
#Lets create a checkpoint for start here next time
data.to_json('./data/10k-xbrl_v2.json')

# Parsing XBRL files

Now that we have all files, we need to find only the tags we need

In [1]:
#importing libraries
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Processing")

#Loading the data
data = pd.read_json('./data/10k-xbrl_v2.json')

In [2]:
def xbrl_tag(url,tags=[]):
    import xml.etree.ElementTree as ET
    import requests
    from bs4 import BeautifulSoup
    import unidecode
    import re
    from nltk.corpus import stopwords

    stops = set(stopwords.words("english"))
    
    text = ''
    #xbrlfile = xbrlfile
    #tree = ET.parse(xbrlfile)
    #root = tree.getroot()
    
    xbrlfile = requests.get(url).text
    root = ET.fromstring(xbrlfile)
    #item = tree.findall('channel/item')
    
    for item in root.findall('.//'):
        #Only will get the tag we need
        for tag in tags:
            if item.tag.find(tag) >= 0:          
                text = BeautifulSoup(item.text,"lxml-xml",).get_text()
                text = unidecode.unidecode(text)
                text = re.sub('\s', ' ',text) 
                text = re.sub("[^a-zA-Z]", " ", text)
                text = text.lower().split()
                text = [w for w in text if not w in stops]
                text = " ".join(text)
    return text            

## Importing Commitment and Contigencies Tags

In [3]:
tags = ['CommitmentsAndContingenciesDisclosure', 'ContingenciesDisclosure','CommitmentsDisclosure']

In [4]:
data['Comitiment'] = data['url'].progress_apply(xbrl_tag,tags=tags)

Processing: 100%|██████████| 100/100 [00:23<00:00,  5.72it/s]


In [5]:
# Show how many companies dont used this tags
print(data['Company'][data['Comitiment'].apply(len) == 0].count(), ' companies don\'t use this tag')

26  companies don't use this tag


##  Importing Income Tax Disclosure

In [6]:
tags = ['IncomeTaxDisclosure']
data['Income_Tax'] = data['url'].progress_apply(xbrl_tag,tags=tags)

Processing: 100%|██████████| 100/100 [00:25<00:00,  5.14it/s]


In [7]:
print(data['Company'][data['Income_Tax'].apply(len) == 0].count(), ' companies don\'t use this tag')

6  companies don't use this tag


In [8]:
#Preview how the text look
data['Comitiment'][3]

'commitments contingencieslegal proceedingssysco engaged various legal proceedings arisen fully adjudicated likelihood loss legal proceedings based definitions within contingency accounting literature ranges remote reasonably possible probable probable reasonably estimable losses accrued based estimates range potential losses associated matters management believe ultimate resolution proceedings either individually aggregate material adverse effect upon consolidated financial position results operations company however final results legal proceedings cannot predicted certainty company failed prevail one legal matters associated realized losses exceed company current estimates range potential losses company consolidated financial position results operations could materially adversely affected future periods commitmentssysco committed aggregate product purchases resale order benefit centralized approach purchasing majority agreements expire within one year however certain agreements terms

In [9]:
#Lets create a checkpoint for start here next time
data.to_json('./data/10k-xbrl_v2.json')

# Dealing with the text

Now we can analize the text

In [1]:
#importing libraries
import pandas as pd

#Loading the data
data = pd.read_json('./data/10k-xbrl_v2.json')

In [2]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [3]:
#Negative words
neg_stemmed = []
url_neg = 'https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/08a269765a6b185d5f3dd522c876043ba9628715/data/opinion-lexicon-English/negative-words.txt'
neg_words = pd.read_fwf(url_neg,encoding='latin-1',skiprows=34,header=None, 
                       names=['negatives'])

for word in neg_words['negatives']:
    neg_stemmed.append(stemmer.stem(word))

neg_stemmed = set(neg_stemmed)

In [4]:
def word_count(data):
    return len(data.split())

In [5]:
def negative_count (data):
    corpus = (" ".join([stemmer.stem(word) \
                                   for word in data.split()])) 
    return len([word for word in corpus.split() if word in neg_stemmed])

In [17]:
#Creating a new collumn for the Commitiment word count 
data['Comitiment word count'] = data['Comitiment'].apply(word_count)
#Creating a new collumn for the Commitiment Negative word count
data['Comitiment Negative words'] = data['Comitiment'].apply(negative_count)
#Creating a new collumn for the ratio
data['Negative Ratio'] = round(data['Comitiment Negative words'] / data['Comitiment word count'],6)

In [19]:
data[['Company','Comitiment word count','Comitiment Negative words','Negative Ratio']].sample(10)

,Company,Comitiment word count,Comitiment Negative words,Negative Ratio
89,CENTENARY INTERNATIONAL CORP,380,23,0.060526
96,GOLDEN EAGLE INTERNATIONAL INC,0,0,NaN
76,SKYLINE CORP,232,20,0.086207
67,"Fenix Parts, Inc.",954,96,0.100629
44,SANFILIPPO JOHN B & SON INC,155,11,0.070968
88,PROCTER & GAMBLE Co,298,15,0.050336
48,SHARPS COMPLIANCE CORP,121,4,0.033058
41,"BofI Holding, Inc.",790,78,0.098734
35,ECOLOCAP SOLUTIONS INC.,32,2,0.062500
61,ELITE GROUP INC.,3,0,0.000000
